# Fine tune with transformers

In [2]:
import transformers
from datasets import load_dataset
import evaluate
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\ict-
[nltk_data]     tyson\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Loading dataset error

Sometimes when loading the dataset wile in GPU enviorment it will give the error that it cannot find the *samsum* dataset. The workaround is to load the dataset while in CPU mode then save it localy or on you drive. After that just switch back to GPU and load the dataset from the local file using *load_from_disk()*

In [3]:
#data = load_dataset('samsum')
#data.save_to_disk('/content/samsum')
data = load_dataset("Samsung/samsum")
metric = evaluate.load('rouge')
model_checkpoints = 'facebook/bart-large-xsum'

## Data tokenization

**max_input** and **max_target** can variy depending on the available computing power

In [4]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

c:\Users\ict-tyson\OneDrive - Association of Accounting Technicians\Desktop\Projects\textsummarization\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ict-tyson\.cache\huggingface\hub\models--facebook--bart-large-xsum. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
data['train'][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [8]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [9]:
tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'dialogue', 'summary'])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\ict-tyson\OneDrive - Association of Accounting Technicians\Desktop\Projects\textsummarization\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 1483.59 examples/s]


## If memory problems

- sample data to smaller sizes

In [32]:
#sample the data
train_sample = tokenize_data['train'].shuffle(seed=42).select(range(500))
validation_sample = tokenize_data['validation'].shuffle(seed=42).select(range(250))
test_sample = tokenize_data['test'].shuffle(seed=42).select(range(50))

In [33]:
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample

## Training process

In [34]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Depending on computing power, batch size can go as low as 1 if necessary

In [35]:
batch_size = 1

In [36]:
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [37]:
#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [38]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=False
    )
#only CUDA available -> fp16=True

In [39]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

C:\Users\ict-tyson\AppData\Local\Temp\ipykernel_15772\2622452287.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [40]:
trainer.train()

 33%|███▎      | 250/750 [1:26:54<2:54:29, 20.94s/it]Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.

AttributeError: 'numpy.float64' object has no attribute 'mid'

## Testing the fine tuned model

In [ ]:
conversation = """
Rann: Hey Harry, how have you been? Long time no see!
Harry: Hey! What a surprise! 
Harry: Yes, you are right, we haven’t seen each other in a long time. How have you been?
Rann: There is an important campaign next week which is keeping me busy otherwise rest is going good in my life. 
Rann: How about you?
Harry: Oh! I just finished a meeting with a very important client of mine and now I finally have some free time. I feel relieved that I’m done with it.
Rann: Good for you then. Hey! Let’s make a plan and catch up with each other after next week. 
Rann: What do you say?
Harry: Sure, why not? Give me a call when you are done with your project.
Rann: Sure, then. 
Rann: Bye, take care.
Harry: Bye buddy.
"""

In [ ]:
model_inputs = tokenizer(conversation,  max_length=max_input, padding='max_length', truncation=True)

In [ ]:
model_inputs

{'input_ids': [0, 50118, 500, 2279, 35, 11468, 3268, 6, 141, 33, 47, 57, 116, 2597, 86, 117, 192, 328, 50118, 29345, 35, 11468, 328, 653, 10, 2755, 328, 1437, 50118, 29345, 35, 3216, 6, 47, 32, 235, 6, 52, 2220, 17, 27, 90, 450, 349, 97, 11, 10, 251, 86, 4, 1336, 33, 47, 57, 116, 50118, 500, 2279, 35, 345, 16, 41, 505, 637, 220, 186, 61, 16, 2396, 162, 3610, 3680, 1079, 16, 164, 205, 11, 127, 301, 4, 1437, 50118, 500, 2279, 35, 1336, 59, 47, 116, 50118, 29345, 35, 5534, 328, 38, 95, 1550, 10, 529, 19, 10, 182, 505, 3653, 9, 4318, 8, 122, 38, 1747, 33, 103, 481, 86, 4, 38, 619, 15126, 14, 38, 17, 27, 119, 626, 19, 24, 4, 50118, 500, 2279, 35, 2497, 13, 47, 172, 4, 11468, 328, 2780, 17, 27, 29, 146, 10, 563, 8, 2916, 62, 19, 349, 97, 71, 220, 186, 4, 1437, 50118, 500, 2279, 35, 653, 109, 47, 224, 116, 50118, 29345, 35, 9136, 6, 596, 45, 116, 12192, 162, 10, 486, 77, 47, 32, 626, 19, 110, 695, 4, 50118, 500, 2279, 35, 9136, 6, 172, 4, 1437, 50118, 500, 2279, 35, 36255, 6, 185, 575, 4, 501

In [ ]:
raw_pred, _, _ = trainer.predict([model_inputs])

***** Running Prediction *****
  Num examples = 1
  Batch size = 1


In [ ]:
raw_pred

array([[    2,     0,     0,     0,   500,  2279,     8,  3268,  2220,
           75,   450,   349,    97,    13,    10,   251,    86,     4,
          248,  2279,    34,    41,   505,   637,   220,   186,     4,
         3268,  1550,    10,   529,    19,    10,  3653,     9,    39,
            8,    16, 15126,    14,    37,    18,   626,    19,    24,
            4,  3268,    40,   492,   248,  2279,    10,   486,    77,
           37,    18,  1550,    19,    39,   695,     4,     2]])

In [ ]:
tokenizer.decode(raw_pred[0])

'</s><s>Harry has just finished a meeting with a very important client of his. Rann will</s>'

In [ ]:
tokenizer.decode(raw_pred[0])

"</s><s><s><s>Rann and Harry haven't seen each other for a long time. Rann has an important campaign next week. Harry finished a meeting with a client of his and is relieved that he's done with it. Harry will give Rann a call when he's finished with his project.</s>"